In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Jun  1 03:45:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install tensorflow_datasets

In [ ]:
! pip install --upgrade pip

! pip install tensorflow

! pip install tensorflow_datasets

     |████████████████████████████████| 1.6MB 13.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

### 1. 데이터(csv) 로드하기

In [ ]:
import pandas as pd
train_data = pd.read_csv("/content/drive/MyDrive/chatbot_transformer/data/cyber_after_preprocess_shuffle_0522.csv",index_col=0)
train_data.head()

,concerns,reply
0,요즘에 여대생들도 욕하나요?,욕은 여대생뿐만이 아니라 상대에게 자신으 분노를 적절하게 표현하는 방법이기도 하세요...
1,가족들이 날 안좋아하나봐. 하고 제가 그렇게 생각하는게 아니라..그냥 진짜로!!!!...,가족들이 나를 안 좋아한다 느끼는군요. 혹 이런 이야기를 상대에게 했을 때 내가 오...
2,안녕하세요!저는 평소 고민이 많았던 감사 본명: 윤 입니다. 저는 아직 14세 미...,그동안 고민되는 마음을 이곳에 오셔서 이야기 나누면서 도움을 받으셨다니 다행이고 ...
3,우울하고 죽고 싶을 정도로 외롭고 공허해요쓸쓸하고조금에 스트레스조차 이젠 몸이 못 ...,너무 힘들어 글 올려주었군요. 조금 있으면 성인이 되고.. 여러가지 마음에 지쳐가...
4,우울증에 걸린 것 같은데 병원에 갈 환경이 아니에요. 혼자 이겨내려고 하는데 좋은 ...,우울증 증상이면 힘든 생각도 많이 나고 굉장히 힘들고 외롭고 어떻게 해야할지 고민되...


In [ ]:
print('챗봇 샘플의 개수 :', len(train_data))

챗봇 샘플의 개수 : 51575


In [ ]:
# 결측값 있는지 확인
print(train_data.isnull().sum())

concerns    0
reply       0
dtype: int64


In [ ]:
# 결측값이 들어있는 행 전체 삭제
train_data = train_data.dropna(axis=0) 

In [ ]:
print(train_data.isnull().sum())

concerns    0
reply       0
dtype: int64


In [ ]:
train_data.columns = ["concerns", "reply"]

In [ ]:
train_data.rename(columns = {0: "concerns", 1: "reply"}, inplace=True)

In [ ]:
train_data = train_data.reset_index(drop=True)

In [ ]:
train_data

,concerns,reply
0,요즘에 여대생들도 욕하나요?,욕은 여대생뿐만이 아니라 상대에게 자신으 분노를 적절하게 표현하는 방법이기도 하세요...
1,가족들이 날 안좋아하나봐. 하고 제가 그렇게 생각하는게 아니라..그냥 진짜로!!!!...,가족들이 나를 안 좋아한다 느끼는군요. 혹 이런 이야기를 상대에게 했을 때 내가 오...
2,안녕하세요!저는 평소 고민이 많았던 감사 본명: 윤 입니다. 저는 아직 14세 미...,그동안 고민되는 마음을 이곳에 오셔서 이야기 나누면서 도움을 받으셨다니 다행이고 ...
3,우울하고 죽고 싶을 정도로 외롭고 공허해요쓸쓸하고조금에 스트레스조차 이젠 몸이 못 ...,너무 힘들어 글 올려주었군요. 조금 있으면 성인이 되고.. 여러가지 마음에 지쳐가...
4,우울증에 걸린 것 같은데 병원에 갈 환경이 아니에요. 혼자 이겨내려고 하는데 좋은 ...,우울증 증상이면 힘든 생각도 많이 나고 굉장히 힘들고 외롭고 어떻게 해야할지 고민되...
...,...,...
51570,쓸데없는 생각과 걱정이 많이 드는데 어떻게 하는것이 좋을까요?,사람은 누구나 벌어지지 않은 일에 대해 걱정을 하기도 한답니다. kim sw님께서도...
51571,드디어 내일이 시험이네요 두근두근!긴장도되고 떨려서 실수할까봐 걱정도 되요 후......,"민트님, 내일이 중간고사라 많이 긴장이 되겠어요. 푹 자고, 아침밥 챙겨 먹고, 너..."
51572,엘베올라가다 문이열리자마자 누가 있더라구요 나올려하다가요 당황을 좀했더니 바로 사장...,때로는 타이밍을 맞추지 못해 늦을 수도 있지만 다음에 만났을 때 먼저 인사하면 될 ...
51573,이제 고등학생이 되는데 하루하루 너무 힘들어서 정신과 상담을 받고싶어요. 부모님은 ...,대부분 정신과를 방문하는 이유는 고통을 줄이기 위해 약물처방을 받기 위해 가게 되는...


In [ ]:
questions = []
for sentence in train_data['concerns']:
    # 구두점에 대해서 띄어쓰기
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [ ]:
answers = []
for sentence in train_data['reply']:
    # 구두점에 대해서 띄어쓰기
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [ ]:
print(questions[:5])
print(answers[:5])

['요즘에 여대생들도 욕하나요 ?', '가족들이 날 안좋아하나봐 .  하고 제가 그렇게 생각하는게 아니라 .  . 그냥 진짜로 !  !  !  !  안좋아해요 .  그렇게 느끼셨군요 ?  하고 답들으면 .  . 꼭 제가 오해한다는 말처럼 느껴져서 마음이 안좋아요 .  저 안좋아해요 .  그냥 .  .  . 진짜로 .  .  . 본인들도 날 사랑할 사람 아무도 없다고 ,  다른 가족들조차 너랑 사실 살기 싫을거다 말하고 다른 친척들도 저를 보고  자기네들은 저처럼 그렇지 않아  다행이라고 말해요 .  .  .  . 저희 언니조차 절 덜 좋아한단 말에 동의해요 .  .  .  .  .   .  .  .  .  .  .  .  .  오ㅐ 절안좋아하는 걸까요', '안녕하세요 ! 저는 평소 고민이 많았던 감사 본명: 윤  입니다 .  저는 아직 14세 미만이라서 계정이 없었는데 로그인없이도 익명으로 고민글을 쓸 수 있는 청소년상담센터가 있어서 잘 이용하고 있어요 .  저의 고민을 들어주신 컴슬러 분들께 감사드립니다 !  앞으로도 로그인 없이 이용가능하면 좋겠습니다 . 성탄절 잘 보내세요', '우울하고 죽고 싶을 정도로 외롭고 공허해요쓸쓸하고조금에 스트레스조차 이젠 몸이 못 버텨요두통만 있었는데 이젠 구역질 ,  호흡곤란까지 와요이제 며칠 뒤면 성인인데 너무 무섭고 미칠 것 같아요요즘 매일 울어요그냥 눈물이 나고 멍 때리고 눈물이 나서 감정을 제어 못하겠어요초등학교 때 자살이 무엇인지 알고 그때부터자살 생각 있고 시도도 해보고 자해 ,  자학도 하는데뭐랄까 이제는 너무 지치고 포기하고 도망가고 싶어요그냥 존재가 없었던 사람이 되고 싶어요잠도 많아졌어요 학교 ,  학원 빼고는 잠만 자요이유를 모르겠어요이대로 영원히 자고 싶어요힘들다고 하면 주변 사람들 중 이해 못 하는 사람이 있어요저는 사람 한 명 대우하기도 너무 힘들어요학생이잖아요학교 다니고 있잖아요 사람이 싫은 건 아니지만 너무 지겹고 지쳐요가족조차도 대우하기 힘들어요', '우울증에 걸린 것 같은데 병원에 갈 환경이 

### 2. 단어 집합 생성

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)

In [ ]:
tokenizer.vocab_size

8199

In [ ]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
print('시작 토큰 번호 :',START_TOKEN)
print('종료 토큰 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE)

시작 토큰 번호 : [8199]
종료 토큰 번호 : [8200]
단어 집합의 크기 : 8201


### 3. 정수 인코딩과 패딩

* 샘플 1개에 대해

In [ ]:
print('임의의 질문 샘플을 정수 인코딩 : {}'.format(tokenizer.encode(questions[20])))

임의의 질문 샘플을 정수 인코딩 : [35, 1749, 2661, 75, 6262, 362, 3653, 3236, 7753, 2273, 5, 195, 17, 4790, 3247, 35, 84, 956, 371, 178, 217, 8177, 8123, 8101, 8179, 8119, 8117, 5159, 207, 40, 3901, 2920, 269, 45]


In [ ]:
sample_string = questions[20]

# encode
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# decode
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [35, 1749, 2661, 75, 6262, 362, 3653, 3236, 7753, 2273, 5, 195, 17, 4790, 3247, 35, 84, 956, 371, 178, 217, 8177, 8123, 8101, 8179, 8119, 8117, 5159, 207, 40, 3901, 2920, 269, 45]
기존 문장: 제가 12살인데 요새 야한게 가끔식 보고싶고 그런데 너무 빠른가요 제가 기독교인인데 괞찮은거에요 ?  아님 자연스러운 거에요 ?


In [ ]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

35 ----> 제가 
1749 ----> 12
2661 ----> 살인
75 ----> 데 
6262 ----> 요새 
362 ----> 야
3653 ----> 한게 
3236 ----> 가끔
7753 ----> 식 
2273 ----> 보고싶
5 ----> 고 
195 ----> 그런데 
17 ----> 너무 
4790 ----> 빠른
3247 ----> 가요 
35 ----> 제가 
84 ----> 기
956 ----> 독
371 ----> 교
178 ----> 인
217 ----> 인데 
8177 ----> �
8123 ----> �
8101 ----> �
8179 ----> �
8119 ----> �
8117 ----> �
5159 ----> 은거
207 ----> 에요
40 ---->  ?  
3901 ----> 아님 
2920 ----> 자연스러운 
269 ----> 거에요
45 ---->  ?


* 전체 데이터에 대해

In [ ]:
MAX_LENGTH = 120

def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)
    
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

In [ ]:
questions, answers = tokenize_and_filter(questions, answers)

In [ ]:
print('질문 데이터의 크기(shape) :', questions.shape)
print('답변 데이터의 크기(shape) :', answers.shape)

질문 데이터의 크기(shape) : (51575, 120)
답변 데이터의 크기(shape) : (51575, 120)


In [ ]:
print(questions[10000])
print(answers[10000])

[8199 4396    8 6902 7332   25 6493 5571    8 2577  580 7083  116 5408
  424   50 1297  172 1796 6396  959 2274  195 1776   26  590 7706   46
   77  454 1410  805  619 3930 3280 3376 2274   47 1685 5197 7975 4790
 2436 1832  170 8200    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[8199 4396    8 5571 3930   44 5408  162 1679  156 1039 2937 7417    7
   12    1 4396    8  590  633   67 5571 3930    8 2044 2606    7 6711
 7975 2332    9  164  337 4423 6144  206  225  544 1513 3295 3383   32
    2 1425  225 2853   46 2064 3204  116 6144  206 5314    4  535  794
 7975 5232  821 2647  339    9 1248 3383    6   18  154    7  597   13
 8200    0    0    0    0    0    0

## 4. 인코더와 디코더의 입력, 그리고 레이블 만들기

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] 
    },
    {
        'outputs': answers[:, 1:]  
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
print(answers[0])
print(answers[:1][:, :-1])
print(answers[:1][:, 1:]) 

[8199 1030   26  136  169  428 6802    8  465 1144  258 1066  442 7975
 4055   19 5355   94 3783 1692  167 2282    1 3306 2222  691   53 1144
   16 2590  599  406 4308 7975  880 4308   30 2507   74 2442 6412 7975
   12    1  325  136  169 3731 2466  226  494 3587    4  678   28 1944
  141  158 8200    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]
[[8199 1030   26  136  169  428 6802    8  465 1144  258 1066  442 7975
  4055   19 5355   94 3783 1692  167 2282    1 3306 2222  691   53 1144
    16 2590  599  406 4308 7975  880 4308   30 2507   74 2442 6412 7975
    12    1  325  136  169 3731 2466  226  494 3587    4  678   28 1944
   141  158 8200    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    

## 5. 트랜스포머 만들기

In [ ]:
! pip install transformers

     |████████████████████████████████| 2.2 MB 14.5 MB/s 
     |████████████████████████████████| 895 kB 51.2 MB/s 
     |████████████████████████████████| 3.3 MB 60.9 MB/s 


In [ ]:
import transformers

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) 
  return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    sines = tf.math.sin(angle_rads[:, 0::2])

    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [ ]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs,
          'mask': padding_mask 
      })

  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)
    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    outputs = self.dense(concat_attention)

    return outputs

In [ ]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, 
          'mask': look_ahead_mask 
      })

  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask 
      })

  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [ ]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  inputs = tf.keras.Input(shape=(None,), name="inputs")

  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) 

  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):

  matmul_qk = tf.matmul(query, key, transpose_b=True)
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  if mask is not None:
    logits += (mask * -1e9)

  attention_weights = tf.nn.softmax(logits, axis=-1)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:
tf.keras.backend.clear_session()

# Hyper-parameters
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8201, 256)
(1, 8201, 256)


### ckeckpoint 저장

In [ ]:
import os

In [ ]:
cd /content/drive/MyDrive/chatbot_transformer

/content/drive/MyDrive/chatbot_transformer


In [ ]:
checkpoint_path = "ckpt_0523/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.save_weights(checkpoint_path.format(epoch=0))

(1, 8201, 256)
(1, 8201, 256)


In [ ]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 150
model.fit(dataset, epochs=EPOCHS, callbacks=[cp_callback])

### 체크포인트 복원

* 만들어진 체크포인트 확인, 마지막 체크포인트 선택

In [ ]:
!ls {checkpoint_dir}

checkpoint			  cp-0075.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0080.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0080.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0085.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0085.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0090.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0090.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0095.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0095.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0100.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0100.ckpt.index
cp-0025.ckpt.data-00000-of-00001  cp-0105.ckpt.data-00000-of-00001
cp-0025.ckpt.index		  cp-0105.ckpt.index
cp-0030.ckpt.data-00000-of-00001  cp-0110.ckpt.data-00000-of-00001
cp-0030.ckpt.index		  cp-0110.ckpt.index
cp-0035.ckpt.data-00000-of-00001  cp-0115.ckpt.data-00000-of-00001
cp-0035.ckpt.index		  cp-0115.ckpt.index
cp-0040.ckpt.data-00000-of-00001  cp-0120.ckpt.data-00000-of-00001
cp-0040.ckpt.index		  cp-0120.ckpt.

In [ ]:

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.load_weights('ckpt_0523/cp-0150.ckpt')

(1, 8201, 256)
(1, 8201, 256)


* 학습 완료 후, 가중치만 따로 저장

In [ ]:
  def get_config(self):
    config = {
    'd_model': self.d_model,
    'warmup_steps': self.warmup_steps,

     }
    return config

## 6. 모델 평가

In [ ]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [ ]:
import re

In [ ]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])
  
  splited = re.split(r'[.?!]', predicted_sentence)
  temp = []
  for i in range(len(predicted_sentence)):
    if predicted_sentence[i] == '.' or predicted_sentence[i] == '?' or predicted_sentence[i] == '!':
      temp.append(predicted_sentence[i])
  # print(splited)
  if len(splited) > 2 :
    predicted_sentence = splited[0]+ temp[0] + splited[-2] + '.'

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))
  

  return predicted_sentence

In [ ]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

## 7. 시현하기

In [ ]:
output = predict("요즘 기분이 너무 안좋아")

Input: 요즘 기분이 너무 안좋아
Output: 무슨 일이 있는걸까요 .  스트레스가 있다면 풀 수 있는 자신만의 방법을 찾아 보는 건 어떨까요 .


In [ ]:
output = predict("요즘 학교생활이 너무 힘드네")

Input: 요즘 학교생활이 너무 힘드네
Output: 학교생활이 힘들군요 .  함께 이야기 나누며 힘을 얻으면 좋겠어요 .


In [ ]:
output = predict("친구들이랑 잘 지내고 싶은데 다가가는 방법을 모르겠어")

Input: 친구들이랑 잘 지내고 싶은데 다가가는 방법을 모르겠어
Output: 친구들과 잘 지내고 싶은 마음이 느껴지네요 .  먼저 친구들에게 다가가서 말을 걸어보면 어떨까요 .


In [ ]:
output = predict("내가 말을 걸면 친구들이 불편해 할까봐 걱정이야")

Input: 내가 말을 걸면 친구들이 불편해 할까봐 걱정이야
Output: 말을 잘 하지 않으면 상대방도 눈치가 보일수 있어요 .  그러니 너무 긴장하지 말고 편하게 말을 건네보세요


In [ ]:
output = predict("반이 바뀌었는데 친구가 하나도 없어")

Input: 반이 바뀌었는데 친구가 하나도 없어
Output: 새학기가 되어서 친구를 사귀기 위해서는 새로운 환경에 적응하는 것이 가장 중요해요 .  학교 생활이 편해야 할 때니 너무 걱정하지 마시고 새로운 친구들과 잘 지낼 수 있도록 해보세요


In [ ]:
output = predict("반배정 나왔는데 친한 친구가 하나도 없어")

Input: 반배정 나왔는데 친한 친구가 하나도 없어
Output: 반배정이 잘 되지 않아서 새학기가 걱정이 되는것 같아요 .  새학기가 되면 새로운 반에 새로운 친구를 사귀어 나가면 된답니다 .


In [ ]:
output = predict("요즘 스트레스가 많아")

Input: 요즘 스트레스가 많아
Output: 요즘 스트레스가 많으신가봐요 .  스트레스를 풀 수 있는 자신만의 해소법을 찾아보면 어떨까요 .


In [ ]:
output = predict("중간고사 공부를 열심히 했는데 시험을 망쳤어")

Input: 중간고사 공부를 열심히 했는데 시험을 망쳤어
Output: 시험을 잘 보고 싶은데 성적이 잘 나오지 않아 많이 속상했을 것 같아요 .  그래도 열심히 공부해서 좋은 결과가 있기를 응원 보낼게요 .


In [ ]:
output = predict("공부를 열심히 했는데 시험을 망쳤어")

Input: 공부를 열심히 했는데 시험을 망쳤어
Output: 열심히 했는데 결과가 좋지 않아 많이 속상하고 마음이 무거울것같아요 .  그래도 남은 과목도 열심히 준비하며 남은 시간동안 최선을 다해서 좋은 결과 있기를 바래요 .


In [ ]:
output = predict("아 시험 망했어")

Input: 아 시험 망했어
Output: 시험을 망쳤다니 너무 속상하겠어요 .  다음 시험에서 좋은 결과가 있기를 바랍니다 .


In [ ]:
output = predict("아니 나 시험 망했어")

Input: 아니 나 시험 망했어
Output: 시험을 망쳤다니 정말 많이 속상하겠어요 .  그래도 시험에서 좋은 결과 있기를 바래요 .


In [ ]:
output = predict("실수를 많이해서 더 속상해")

Input: 실수를 많이해서 더 속상해
Output: 에고 실수를 해서 너무 속상하고 마음이 아프겠어요 .  너무 자책하지 마시고 다음에는 꼭 좋은 결과가 있길 바래요 .


In [ ]:
output = predict("정말 열심히 공부했는데 성적이 떨어졌어")

Input: 정말 열심히 공부했는데 성적이 떨어졌어
Output: 열심히 공부해서 성적이 떨어졌다면 더 기운이 빠질 수 있을 것 같아요 .  그래도 열심히 공부해서 좋은 결과 있기를 응원할게요 .


In [ ]:
output = predict("난 쓸모없는 사람인 것 같아")	

Input: 난 쓸모없는 사람인 것 같아
Output: 쓸모없는 사람이라고 생각하니 너무 힘들고 지치지요 .  조금 더 자세히 이야기를 나누어보면 좋겠어요 .


In [ ]:
output = predict("친구들에게 따돌림을 당하고 있어")

Input: 친구들에게 따돌림을 당하고 있어
Output: 따돌림을 당하고 있는 상황이라 너무 외롭고 견디기 어려웠을 듯 해요 .  혼자 끙끙 앓지 말고 누군가와 이야기를 나눠보고  도움을 청할 수 있는 사람에게 도움을 청하면 좋겠어요 .


In [ ]:
output = predict("친구들에게 괴롭힘만 당하느니 죽고 싶어")

Input: 친구들에게 괴롭힘만 당하느니 죽고 싶어
Output: 괴롭힘을 당하다니 얼마나 괴롭고 힘들까요 .  혼자서 해결하기 어렵다면 부모님이나 담임선생님과 상의해서 도움을 받아보면 좋겠어요 .


In [ ]:
output = predict("요즘 공부에 너무 집중이 안되는데 어떻게 해야해")

Input: 요즘 공부에 너무 집중이 안되는데 어떻게 해야해
Output: 공부를 해야 하는 이유는 뭘까요 ?  해야 하는 이유를 알고 있다면 동기부여가 필요할 것 같아요 .


In [ ]:
output = predict("과거에 안좋은 기억들 땜에 괴로워요")

Input: 과거에 안좋은 기억들 땜에 괴로워요
Output: 때론 과거의 안좋은 일이 생각나면 마음속에 남아있게 되시는것 같아요 .  과거의 안좋은 일이 생각나고 후회되고 자책을 하게 되는것 같아요 .


In [ ]:
output = predict("날씨가 흐려서 우울해졌어요 위로받고 싶어요")

Input: 날씨가 흐려서 우울해졌어요 위로받고 싶어요
Output: 에고 .  .  얼마나 힘들고 ,  슬프고 화도 나고 기운도 없을것 같아요   이런 속상한 마음을 털어버리시고 힘내세요


In [ ]:
output = predict("좋아하는 남자친구가 있는데 고백할 용기가 나지 않아")

Input: 좋아하는 남자친구가 있는데 고백할 용기가 나지 않아
Output: 좋아하는 남자친구가 있는데 고백 중이신가봐요 .  고백은 문자나 톡 등을 통해 표현해봐도 좋을 것 같아요 .  고백하기 전에 문자로라도 꼭 해보세요


In [ ]:
output = predict("수면부족으로 잠을 못자서 너무 피곤해")

Input: 수면부족으로 잠을 못자서 너무 피곤해
Output: 잠을 못자면 다음날 피곤할 것 같아요 .  잠이 안 올 때는 억지로 자려고 하지 말고 누워서 잠을 자려고 하지 말고 누워서 하루를 보내면 좋겠네요 .


In [ ]:
output = predict("요즘 공부에 너무 집중이 안되는데 어떻게 해야해?")

Input: 요즘 공부에 너무 집중이 안되는데 어떻게 해야해?
Output: 공부를 해야 하는 이유는 뭘까요 ?  해야 하는 이유를 알고 있다면 동기부여가 생기지 않을까요 ?


In [ ]:
output = predict("살고 싶지 않아 죽고싶어")

Input: 살고 싶지 않아 죽고싶어
Output: 죽고싶은 마음이 들 정도로 많이 힘들어보이네요 .  죽고싶은 마음이 들 정도로 힘든 일이 있었나봐요 .
